# Algoritmo A* (tree search e graph search)

Creazione delle connessioni e delle euristiche

In [1]:
connections = {} # distanze reali
h = {} # distanze in linea d'aria (euristiche)
connections['Arad'] = [['Sibiu', 140], ['Timisoara', 118], ['Zerind',  75]]
connections['Bucarest'] = [['Fagaras', 211], ['Giurgiu', 90], ['Pitesti', 101], ['Urziceni', 85]]
connections['Craiova'] = [['Drobeta', 120], ['Pitesti', 138], ['Rimnicu Vilcea', 146]]
connections['Drobeta'] = [['Craiova', 120], ['Mehadia', 75]]
connections['Eforie'] = [['Hirsova', 86]]
connections['Fagaras'] = [['Bucarest', 211], ['Sibiu', 99]]
connections['Giurgiu'] = [['Bucarest', 90]]
connections['Hirsova'] = [['Eforie', 86], ['Urziceni', 98]]
connections['Iasi'] = [['Neamt', 87], ['Vaslui', 92]]
connections['Lugoj'] = [['Mehadia', 70], ['Timisoara', 111]]
connections['Mehadia'] = [['Drobeta', 75], ['Lugoj', 70]]
connections['Neamt'] = [['Iasi', 87]]
connections['Oradea'] = [['Sibiu', 151], ['Zerind', 71]]
connections['Pitesti'] = [['Bucarest', 101], ['Craiova', 138], ['Rimnicu Vilcea', 97]]
connections['Rimnicu Vilcea'] = [['Craiova', 146], ['Pitesti', 97], ['Sibiu', 80]]
connections['Sibiu'] = [['Arad', 140], ['Fagaras', 99], ['Oradea', 151], ['Rimnicu Vilcea', 80]]
connections['Timisoara'] = [['Arad', 118], ['Lugoj', 111]]
connections['Urziceni'] = [['Bucarest', 85], ['Hirsova', 98], ['Vaslui', 142]]
connections['Vaslui'] = [['Iasi', 92], ['Urziceni', 142]]
connections['Zerind'] = [['Arad', 75], ['Oradea', 71]]
h['Arad'] = 366
h['Bucarest'] = 0
h['Craiova'] = 160
h['Drobeta'] = 242
h['Eforie'] = 161
h['Fagaras'] = 176
h['Giurgiu'] = 77
h['Hirsova'] = 151
h['Iasi'] = 226
h['Lugoj'] = 244
h['Mehadia'] = 241
h['Neamt'] = 234
h['Oradea'] = 380
h['Pitesti'] = 100
h['Rimnicu Vilcea'] = 193
h['Sibiu'] = 253
h['Timisoara'] = 329
h['Urziceni'] = 80
h['Vaslui'] = 199
h['Zerind'] = 374

Creazione degli stati e dei nodi

In [2]:
class State:

    # COSTRUTTORE
    def __init__(self, name = None):
        if name == None:
            self.name = self.getInitialState() # Stato iniziale
        else:
            self.name = name

    # STATO INIZIALE
    def getInitialState(self):
        initialState = 'Arad'
        return initialState

    # SUCCESSORI
    def successorFunction(self):
        return connections[self.name] # E' una dictonary

    # CONTROLLA SE E' UNO STATO OBIETTIVO
    def checkGoalState(self):
        return self.name == 'Bucarest'

class Node:

    # COSTRUTTORE
    # state --> stato
    # parent --> genitore
    # f --> funzione di valutazione
    # partial_path --> costo del cammino parziale
    def __init__(self, state, parent, f, partial_path):
        self.state = state
        self.depth = 0 # profondita'
        self.children = [] # array dei suoi figli
        self.parent = parent
        self.heuristic = f # euristica
        self.partial_path = partial_path

    # AGGIUNGI FIGLIO
    def addChild(self, child):
        self.children.append(child)
        child.parent = self # metti come genitore del figlio se stesso
        child.depth = self.depth + 1 # il figlio si trova piu' profondo di 1

    # STAMPA IL CAMMINO
    def printPath(self):
        if self.parent != None:
            # Passo ricorsivo
            self.parent.printPath()
            print(' -> ', end='')
        print(self.state.name, end='') # Stampa del nome dello stato



Definizione della frontiera. La frontiera viene implementata con una lista ordinata.

In [3]:
class Element:
    value = None # Funzione di valutazione
    node = None # Nodo
    next = None # Prossimo elemento

    # COSTRUTTORE
    def __init__(self, value, node):
        self.value = value
        self.node = node
        self.next = None

class Fringe:

    # COSTRUTTORE
    def __init__(self):
        self.__head = None # Elemento in testa della frontiera
        self.__tail = None # Elemento in coda della frontiera

    # CONTROLLO FRONTIERA VUOTA
    def empty_fringe(self):
        return self.__head == None

    # AGGIUNGE UN NODO IN FRONTIERA
    def add(self, newNode):
        if self.__head == None:
            # Aggiungere primo elemento della frontiera
            self.__head = newNode
            self.__tail = self.__head
            newNode.next = None
        elif newNode.value < self.__head.value:
            # Aggiungere un elemento in testa alla frontiera
            newNode.next = self.__head
            self.__head = newNode
        elif newNode.value > self.__tail.value:
            # Aggiungere un elemento in coda alla frontiera
            self.__tail.next = newNode
            self.__tail = newNode
            newNode.next = None
        else:
            # Aggiungere un elemento in frontiera
            temp = self.__head # variabile per lo scorrimento
            # temp --> temp.next
            # temp --> newNode --> temp.next
            while temp.next != None and (newNode.value > temp.value):
                temp = temp.next # passa al prossimo elemento
            newNode.next = temp.next
            temp.next = newNode

    # Estrazione del primo elemento in frontiera
    def estract(self):
        out = self.__head # variabile di output
        if out != None:
            self.__head = self.__head.next # passa al prossimo elemento
            out.next = None
        return out

    def print(self):
        print('Fringe: ',end='')
        temp = self.__head # variabile per lo scorrimento
        while temp != None:
            if temp.next == None:
                print(self.node.state.name)
            else:
                print(self.node.state.name, '->', end=' ')
            temp = temp.next

## A* - Tree search

In [4]:
def A_star_tree():

    fringe = Fringe() # Frontiera
    initialState = State() # Stato iniziale
    heuristic = h[initialState.name] # Euristica
    root = Node(initialState, None, heuristic, 0) # Nodo radice
    # Metti il nodo radice in frontiera
    element = Element(heuristic, root)
    fringe.add(element)
    print('Inizio algoritmo')
    while not fringe.empty_fringe():
        # Estrai il primo nodo dalla radice
        estracted = fringe.estract()
        currentNode = estracted.node
        print("Nodo estratto: ", currentNode.state.name)
        # Verifica che il nodo scelto sia il nodo obiettivo
        if currentNode.state.checkGoalState():
            # Nodo obiettivo trovato: stampa il cammino
            print('Stato obiettivo raggiunto')
            print('-------------------------')
            print('Soluzione: ', end='')
            currentNode.printPath()
            break
        # Metti i suoi successori in frontiera
        childStates = currentNode.state.successorFunction()
        #  childStates(nome,costo)
        for (childState, distance) in childStates:
            # Calcola la funzione di valutazione
            g = currentNode.partial_path + distance # Funzione di costo
            heuristic = h[childState] # Funzione euristica
            f = g + heuristic # Funzione di valutazione
            # Crea il nodo figlio e aggiungilo in frontiera
            childNode = Node(State(childState), currentNode, f, g) # Crea un nuovo nodo
            currentNode.addChild(childNode) # Aggiungi il nodo ai figli (Facoltativo)
            element = Element(childNode.heuristic, childNode) # Crea nuovo elemento
            fringe.add(element) # Aggiungi l'elemento in frontiera

In [5]:
A_star_tree()

Inizio algoritmo
Nodo estratto:  Arad
Nodo estratto:  Sibiu
Nodo estratto:  Rimnicu Vilcea
Nodo estratto:  Fagaras
Nodo estratto:  Timisoara
Nodo estratto:  Pitesti
Nodo estratto:  Bucarest
Stato obiettivo raggiunto
-------------------------
Soluzione: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucarest

## A* graph search

In [6]:
def A_star_graph():

    fringe = Fringe() # Frontiera
    initialState = State() # Stato iniziale
    heuristic = h[initialState.name] # Euristica
    root = Node(initialState, None, heuristic, 0) # Nodo radice
    # Metti il nodo radice in frontiera
    element = Element(heuristic, root)
    fringe.add(element)
    close = [] # Close
    print('Inizio algoritmo')
    while not fringe.empty_fringe():
        # Estrai il primo nodo dalla radice
        estracted = fringe.estract()
        currentNode = estracted.node
        print("Nodo estratto: ", currentNode.state.name)
        # Verifica che il nodo scelto sia il nodo obiettivo
        if currentNode.state.checkGoalState():
            # Nodo obiettivo trovato: stampa il cammino
            print('Stato obiettivo raggiunto')
            print('-------------------------')
            print('Soluzione: ', end='')
            currentNode.printPath()
            break
        # Controlla che il nodo non sia in frontiera
        if currentNode.state.name not in close:
            close.append(currentNode.state.name) # aggiungi lo stato nella close
            # Metti i suoi successori in frontiera
            childStates = currentNode.state.successorFunction()
            #  childStates(nome,costo)
            for (childState, distance) in childStates:
                # Calcola la funzione di valutazione
                g = currentNode.partial_path + distance # Funzione di costo
                heuristic = h[childState] # Funzione euristica
                f = g + heuristic # Funzione di valutazione
                # Crea il nodo figlio e aggiungilo in frontiera
                childNode = Node(State(childState), currentNode, f, g) # Crea un nuovo nodo
                currentNode.addChild(childNode) # Aggiungi il nodo ai figli (Facoltativo)
                # Aggiungi il figlio se non sta in close
                if childNode.state.name not in close:
                    element = Element(childNode.heuristic, childNode) # Crea nuovo elemento
                    fringe.add(element) # Aggiungi l'elemento in frontiera

In [7]:
A_star_graph()

Inizio algoritmo
Nodo estratto:  Arad
Nodo estratto:  Sibiu
Nodo estratto:  Rimnicu Vilcea
Nodo estratto:  Fagaras
Nodo estratto:  Timisoara
Nodo estratto:  Pitesti
Nodo estratto:  Bucarest
Stato obiettivo raggiunto
-------------------------
Soluzione: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucarest